In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, optimizers, losses, models
import matplotlib.pyplot as plt

# Loading Data

In [4]:
class PhaseLayer(layers.Layer):
    def __init__(self, rng=np.random.RandomState(23456),units=512, input_dim=512, number_of_phases=4):
        super(PhaseLayer, self).__init__()
        self.nslices = number_of_phases
        self.units = units
        self.input_dim = input_dim
        self.rng = rng
    
        self.w = tf.Variable(self.initial_alpha(), name="w", trainable=True)
        self.b = tf.Variable(self.initial_beta(), name="b", trainable=True)
    
    def initial_alpha(self):
        shape = (self.nslices, self.input_dim, self.units)
        alpha_bound = np.sqrt(6. / np.prod(shape[-2:]))
        alpha = np.asarray(
            self.rng.uniform(low=-alpha_bound, high=alpha_bound, size=shape),
            dtype=np.float32
        )
        return tf.convert_to_tensor(alpha, dtype=tf.float32)
    
    def initial_beta(self):
        return tf.zeros((self.nslices, self.units), dtype=tf.float32)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [11]:
class PFNN(tf.keras.Model):
    def __init__(self, input_dim=1, output_dim=1, dropout=0.3, **kwargs):
        super(PFNN ,self).__init__(**kwargs)
        self.nslices = 4
        self.input_dim=input_dim
        self.output_dim=output_dim
    
        self.dropout0 = layers.Dropout(dropout)
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)
    
        self.activation = layers.ELU()
    
        self.layer0 = PhaseLayer(input_dim=input_dim)
        self.layer1 = PhaseLayer()
        self.layer2 = PhaseLayer(units=output_dim)

    def call(self, inputs):
        pscale = self.nslices * inputs[:,-1]
        pamount = pscale % 1.0
    
        pindex_1 = tf.cast(pscale, 'int32') % self.nslices
        pindex_0 = (pindex_1-1) % self.nslices
        pindex_2 = (pindex_1+1) % self.nslices
        pindex_3 = (pindex_1+2) % self.nslices
    
        bamount = tf.expand_dims(pamount, 1)
        wamount = tf.expand_dims(bamount, 1)

        def cubic(y0, y1, y2, y3, mu):
            return (
           (-0.5*y0+1.5*y1-1.5*y2+0.5*y3)*mu*mu*mu + 
           (y0-2.5*y1+2.0*y2-0.5*y3)*mu*mu + 
           (-0.5*y0+0.5*y2)*mu +
           (y1))
    
        W0 = cubic(
            tf.nn.embedding_lookup(self.layer0.w, pindex_0), 
            tf.nn.embedding_lookup(self.layer0.w, pindex_1), 
            tf.nn.embedding_lookup(self.layer0.w, pindex_2), 
            tf.nn.embedding_lookup(self.layer0.w, pindex_3), 
            wamount)
        W1 = cubic(
            tf.nn.embedding_lookup(self.layer1.w, pindex_0), 
            tf.nn.embedding_lookup(self.layer1.w, pindex_1), 
            tf.nn.embedding_lookup(self.layer1.w, pindex_2), 
            tf.nn.embedding_lookup(self.layer1.w, pindex_3), 
            wamount)
        W2 = cubic(
            tf.nn.embedding_lookup(self.layer2.w, pindex_0), 
            tf.nn.embedding_lookup(self.layer2.w, pindex_1), 
            tf.nn.embedding_lookup(self.layer2.w, pindex_2), 
            tf.nn.embedding_lookup(self.layer2.w, pindex_3), 
            wamount)
        
        b0 = cubic(
            tf.nn.embedding_lookup(self.layer0.b, pindex_0), 
            tf.nn.embedding_lookup(self.layer0.b, pindex_1), 
            tf.nn.embedding_lookup(self.layer0.b, pindex_2), 
            tf.nn.embedding_lookup(self.layer0.b, pindex_3), 
            bamount)
        b1 = cubic(
            tf.nn.embedding_lookup(self.layer1.b, pindex_0),
            tf.nn.embedding_lookup(self.layer1.b, pindex_1),
            tf.nn.embedding_lookup(self.layer1.b, pindex_2),
            tf.nn.embedding_lookup(self.layer1.b, pindex_3),
            bamount)
        b2 = cubic(
            tf.nn.embedding_lookup(self.layer2.b, pindex_0),
            tf.nn.embedding_lookup(self.layer2.b, pindex_1),
            tf.nn.embedding_lookup(self.layer2.b, pindex_2),
            tf.nn.embedding_lookup(self.layer2.b, pindex_3),
            bamount)
        
        H0 = inputs[:, :-1]
        H1 = self.activation(tf.matmul(self.dropout0(H0), W0) + b0)
        H2 = self.activation(tf.matmul(self.dropout0(H1), W1) + b1)
        H3 = tf.matmul(self.dropout2(H2), W2) + b2
    
        return H3
    
    def save_checkpoint(self, direction):
        W0 = self.layer0.w.numpy()
        W1 = self.layer1.w.numpy()
        W2 = self.layer2.w.numpy()
        
        b0 = self.layer0.b.numpy()
        b1 = self.layer1.b.numpy()
        b2 = self.layer2.b.numpy()
        np.savez_compressed(direction + "layer0", weights=W0, bias=b0)
        np.savez_compressed(direction + "layer1", weights=W1, bias=b1)
        np.savez_compressed(direction + "layer2", weights=W0, bias=b2)
    def load_checkpoint(self, direction):
        layer0 = np.load(direction + "layer0.npz")
        self.layer0.w = tf.Variable(layer0["weights"], name="w", trainable=True)
        self.layer0.b = tf.Variable(layer0["bias"], name="b", trainable=True)
        del layer0
        layer1 = np.load(direction + "layer1.npz")
        self.layer1.w = tf.Variable(layer1["weights"], name="w", trainable=True)
        self.layer1.b = tf.Variable(layer1["bias"], name="b", trainable=True)
        del layer1
        layer2 = np.load(direction + "layer2.npz")
        self.layer2.w = tf.Variable(layer2["weights"], name="w", trainable=True)
        self.layer2.b = tf.Variable(layer2["bias"], name="b", trainable=True)
        del layer2
        

In [12]:
pfnn = PFNN(input_dim=342, output_dim=311)

In [13]:
print(pfnn.layer0.w)

<tf.Variable 'w:0' shape=(4, 342, 512) dtype=float32, numpy=
array([[[ 3.9915387e-03, -2.2735364e-04, -5.3028404e-03, ...,
         -4.9895314e-03, -4.4215922e-03, -2.0417425e-05],
        [-5.5587054e-03, -1.8462978e-03,  3.2985634e-03, ...,
         -4.4966852e-03, -3.3647593e-03,  4.7771307e-03],
        [ 2.1949239e-04, -5.3447997e-03, -5.5682147e-03, ...,
         -2.2389719e-03,  2.5809940e-03, -3.4271574e-03],
        ...,
        [-3.2437986e-03,  3.1618730e-04, -2.3515772e-03, ...,
          1.1554204e-03,  8.1763515e-04, -1.5074635e-03],
        [ 3.0957342e-03, -4.8389900e-03, -1.1360914e-03, ...,
          3.1572573e-03, -3.4793492e-03, -2.8898085e-03],
        [-2.7632623e-03,  4.5806213e-05,  3.8866622e-03, ...,
         -2.5057618e-03,  4.4323169e-03,  5.7017626e-03]],

       [[ 1.5719665e-03, -5.8694543e-05, -2.5987346e-03, ...,
         -4.6469932e-03, -4.4354592e-03, -5.2308724e-03],
        [-5.4492182e-03,  1.4326838e-03,  2.5002426e-03, ...,
          4.2201784e-0

In [14]:
pfnn.load_checkpoint("")

In [15]:
print(pfnn.layer0.w)

<tf.Variable 'w:0' shape=(4, 342, 512) dtype=float32, numpy=
array([[[-2.1874236e-02,  1.2708489e-02, -3.3418018e-02, ...,
         -4.8815301e-03, -4.9065217e-02, -6.1113413e-02],
        [-2.8227666e-02, -2.9756939e-03, -2.9001614e-02, ...,
          3.3653039e-02, -1.5175923e-02, -8.6393934e-03],
        [ 1.9796189e-02,  1.3329241e-02, -4.0740939e-03, ...,
          1.5201372e-02, -1.0370106e-02, -1.5276201e-02],
        ...,
        [-7.6742661e-03,  1.4448602e-02,  3.0649295e-02, ...,
          6.5858997e-02,  1.0008376e-02, -4.1618054e-03],
        [-1.9641727e-02, -3.4242377e-02, -8.1533296e-03, ...,
         -6.1956126e-02, -1.5728477e-02, -4.7052041e-02],
        [ 3.1691357e-02, -2.0134596e-02,  3.2153707e-02, ...,
         -4.4390369e-02,  2.7882820e-02,  7.0416592e-03]],

       [[ 1.7522346e-02, -1.6397582e-02, -4.8347428e-03, ...,
         -2.8847229e-02, -2.7360739e-02,  8.6625041e-03],
        [ 1.2748019e-02,  2.2621073e-02, -7.5318590e-03, ...,
          3.7453740e-0